### Clean Library Branches

_Combines the library branch circulation, computer session, and visitor datasets and standardizes branch names, locations, and hours. The output is two clean CSV files: `branches.csv` and `branch_metrics.csv`._

#### Setup

In [1]:
# Import packages from Python standard library
import numpy as np
import pandas as pd
import re
from pathlib import Path

#### Combine Metric Files

In [2]:
# Generate path to directory holding raw datasets
raw_dir = Path.joinpath(Path.cwd().parent, Path("data/raw"))

# Combine datasets into single DataFrame
df = None
for dataset in ("circulation", "computer_sessions", "visitors"):
    data_dir = raw_dir / dataset
    for pth in Path.glob(data_dir, "*.csv"):
        fname = pth.as_posix().split("/")[-1]
        year_df = pd.read_csv(pth)
        year_df['TYPE'] = dataset.upper()
        year_df["YEAR"] = int("".join(c for c in fname if c.isdigit()))
        df = year_df if df is None else pd.concat([df, year_df])

# Preview DataFrame
df.head()

,LOCATION,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,...,DECEMBER,YTD,TYPE,YEAR,BRANCH,ADDRESS,CITY,ZIP,Location,ZIP CODE
0,Albany Park,9381.0,8156.0,10619.0,8660.0,7892.0,9795.0,10766.0,10505.0,8285.0,...,7982.0,111168.0,CIRCULATION,2018,NaN,NaN,NaN,NaN,NaN,NaN
1,Altgeld,327.0,283.0,354.0,274.0,384.0,383.0,320.0,327.0,362.0,...,421.0,4469.0,CIRCULATION,2018,NaN,NaN,NaN,NaN,NaN,NaN
2,Archer Heights,5459.0,5097.0,6391.0,5184.0,4415.0,4988.0,5555.0,5153.0,4951.0,...,4856.0,62312.0,CIRCULATION,2018,NaN,NaN,NaN,NaN,NaN,NaN
3,Austin,0.0,884.0,1266.0,1059.0,972.0,1170.0,1310.0,1166.0,809.0,...,848.0,11569.0,CIRCULATION,2018,NaN,NaN,NaN,NaN,NaN,NaN
4,Austin-Irving,9333.0,7887.0,10465.0,8948.0,7946.0,9571.0,9891.0,9146.0,8366.0,...,8146.0,108252.0,CIRCULATION,2018,NaN,NaN,NaN,NaN,NaN,NaN


#### Standardize Metric Library Branch Names

In [3]:
# Define function to select correct branch name and
# remove extra spaces and asterisks.

# NOTE: Beginning in 2020, the name of the column holding
# the library branch name changed from "LOCATION" to "BRANCH"
def correct_name(s: pd.Series):
    name = s["LOCATION"] if s["YEAR"] < 2020 else s["BRANCH"]
    if name is np.nan:
        return None
    return ' '.join(w for w in re.split('\*|\s', name) if w)

# Apply function to DataFrame to generate new column
df['BRANCH_NAME'] = df.apply(correct_name, axis="columns")

In [4]:
# Preview branch names
df['BRANCH_NAME'].sort_values().unique().tolist()

['Albany Park',
 'Altgeld',
 'Archer Heights',
 'Austin',
 'Austin-Irving',
 'Auto-Renewals',
 'Avalon',
 'Back of the Yards',
 'Beverly',
 'Bezazian',
 'Blackstone',
 'Brainerd',
 'Brighton Park',
 'Bucktown-Wicker Park',
 'Budlong Woods',
 'Canaryville',
 'Chicago Bee',
 'Chicago Lawn',
 'Chinatown',
 'Clearing',
 'Coleman',
 'Daley, Richard J.',
 'Daley, Richard J. -Bridgeport',
 'Daley, Richard J.- Bridgeport',
 'Daley, Richard J.-Bridgeport',
 'Daley, Richard M.',
 'Daley, Richard M. -W. Humboldt',
 'Daley, Richard M.- W Humboldt',
 'Daley, Richard M.-W Humboldt',
 'Douglass',
 'Downloadable Media',
 'Dunning',
 'Edgebrook',
 'Edgewater',
 'Gage Park',
 'Galewood Mont-Clare',
 'Galewood-Mont Clare',
 'Garfield Ridge',
 'Greater Grand Crossing',
 'HAROLD WASHINGTON LIBRARY CENTER',
 'Hall',
 'Harold Washington Library Center',
 'Harold Washtington Library Center',
 'Hegewisch',
 'Humboldt Park',
 'Independence',
 'Itivia Renewal',
 'Itivia Renewals',
 'Jefferson Park',
 'Jeffery Ma

In [5]:
# Define crosswalk to correct erroneous branch spellings leading to dupes
crosswalk = {
    "Daley, Richard J. - Bridgeport": [
        'Daley, Richard J.',
        'Daley, Richard J. -Bridgeport',
        'Daley, Richard J.- Bridgeport',
        'Daley, Richard J.-Bridgeport'
    ],
    "Daley, Richard M. - W. Humboldt": [
        'Daley, Richard M.',
        'Daley, Richard M. -W. Humboldt',
        'Daley, Richard M.- W Humboldt',
        'Daley, Richard M.-W Humboldt'
    ],
    "Galewood-Mont Clare": [
        "Galewood Mont-Clare"
    ],
    "Harold Washington Library Center": [
        "HAROLD WASHINGTON LIBRARY CENTER",
        "Harold Washtington Library Center"
    ],
    "Itivia Renewals": [
        "Itivia Renewal"
    ],
    "Legler Regional": [
        "Legler"
    ],
    "Patron Initiated Renewals (Automated Phone)": [
        "Patron Initiated renewals (automated phone)",
    ],
    "Patron Initiated Renewals (Online)": [
        "Patron Initiated renewals (online)"
    ],
    "Sulzer Regional": [
        "Sulzer Regional Library"
    ],
    "Talking Books and Braille Downloadables": [
        "Talking Book and Braille downloadable",
        "Talking Books"
    ],
    "Woodson Regional": [
        "Woodson Regional Library"
    ]
}

In [6]:
# Correct branch name spellings
for correct_name, misspellings in crosswalk.items():
    df["BRANCH_NAME"] = df["BRANCH_NAME"].apply(lambda n: correct_name if n in misspellings else n)

In [7]:
# Filter out rows that don't pertain to physical branches
invalid_branches = [
    'Downloadable Media',
    'Online Renewals',
    'Itivia Renewals',
    'eBooks/Downloadable Media',
    'Talking Books and Braille Downloadables',
    'Auto-Renewals',
    'Patron Initiated Renewals (Automated Phone)',
    'Patron Initiated Renewals (Online)',
    'YOUmedia',
    'Total'
]
df = df.query("BRANCH_NAME not in @invalid_branches")
df.head()

,LOCATION,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,...,YTD,TYPE,YEAR,BRANCH,ADDRESS,CITY,ZIP,Location,ZIP CODE,BRANCH_NAME
0,Albany Park,9381.0,8156.0,10619.0,8660.0,7892.0,9795.0,10766.0,10505.0,8285.0,...,111168.0,CIRCULATION,2018,NaN,NaN,NaN,NaN,NaN,NaN,Albany Park
1,Altgeld,327.0,283.0,354.0,274.0,384.0,383.0,320.0,327.0,362.0,...,4469.0,CIRCULATION,2018,NaN,NaN,NaN,NaN,NaN,NaN,Altgeld
2,Archer Heights,5459.0,5097.0,6391.0,5184.0,4415.0,4988.0,5555.0,5153.0,4951.0,...,62312.0,CIRCULATION,2018,NaN,NaN,NaN,NaN,NaN,NaN,Archer Heights
3,Austin,0.0,884.0,1266.0,1059.0,972.0,1170.0,1310.0,1166.0,809.0,...,11569.0,CIRCULATION,2018,NaN,NaN,NaN,NaN,NaN,NaN,Austin
4,Austin-Irving,9333.0,7887.0,10465.0,8948.0,7946.0,9571.0,9891.0,9146.0,8366.0,...,108252.0,CIRCULATION,2018,NaN,NaN,NaN,NaN,NaN,NaN,Austin-Irving


In [8]:
# Drop rows without any branch name
# NOTE: These rows were used to capture count totals
df = df.query("BRANCH_NAME == BRANCH_NAME")

#### Restructure Metrics Dataset

In [9]:
# Drop unused columns ahead of restructuring
df = df.drop(columns=[
    'LOCATION',
    'BRANCH',
    'ADDRESS',
    'CITY',
    'ZIP',
    'Location',
    'ZIP CODE',
    'YTD'
])
df.head()

,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,TYPE,YEAR,BRANCH_NAME
0,9381.0,8156.0,10619.0,8660.0,7892.0,9795.0,10766.0,10505.0,8285.0,10098.0,9029.0,7982.0,CIRCULATION,2018,Albany Park
1,327.0,283.0,354.0,274.0,384.0,383.0,320.0,327.0,362.0,525.0,509.0,421.0,CIRCULATION,2018,Altgeld
2,5459.0,5097.0,6391.0,5184.0,4415.0,4988.0,5555.0,5153.0,4951.0,5426.0,4837.0,4856.0,CIRCULATION,2018,Archer Heights
3,0.0,884.0,1266.0,1059.0,972.0,1170.0,1310.0,1166.0,809.0,1162.0,923.0,848.0,CIRCULATION,2018,Austin
4,9333.0,7887.0,10465.0,8948.0,7946.0,9571.0,9891.0,9146.0,8366.0,9435.0,9118.0,8146.0,CIRCULATION,2018,Austin-Irving


In [10]:
# Unpivot dataset from wide to long format
months = {
    "JANUARY": 1,
    "FEBRUARY": 2,
    "MARCH": 3,
    "APRIL": 4,
    "MAY": 5, 
    "JUNE": 6,
    "JULY": 7,
    "AUGUST": 8,
    "SEPTEMBER": 9,
    "OCTOBER": 10,
    "NOVEMBER": 11,
    "DECEMBER": 12
}
melted = df.melt(
    id_vars=['BRANCH_NAME', 'YEAR', 'TYPE'],
    value_vars=list(months.keys()),
    var_name="MONTH")

# Preview DataFrame
melted.head()

,BRANCH_NAME,YEAR,TYPE,MONTH,value
0,Albany Park,2018,CIRCULATION,JANUARY,9381.0
1,Altgeld,2018,CIRCULATION,JANUARY,327.0
2,Archer Heights,2018,CIRCULATION,JANUARY,5459.0
3,Austin,2018,CIRCULATION,JANUARY,0.0
4,Austin-Irving,2018,CIRCULATION,JANUARY,9333.0


In [11]:
# Pivot DataFrame to have the following structure:
# "location,year,month,circulation,computer sessions,visitors"
pivoted = melted.pivot(
    index=['BRANCH_NAME', 'YEAR', 'MONTH'],
    columns="TYPE",
    values="value")

# Preview DataFrame
pivoted.head()

TYPE                       CIRCULATION  COMPUTER_SESSIONS  VISITORS
BRANCH_NAME YEAR MONTH                                             
Albany Park 2011 APRIL          9344.0             1841.0    9300.0
                 AUGUST        11306.0             2434.0   11078.0
                 DECEMBER       9934.0             1594.0    9609.0
                 FEBRUARY       7023.0             1296.0   10500.0
                 JANUARY        8427.0             1666.0    9604.0

In [12]:
# Reset index
final_df = pivoted.reset_index()

# Preview DataFrame
final_df

TYPE,BRANCH_NAME,YEAR,MONTH,CIRCULATION,COMPUTER_SESSIONS,VISITORS
0,Albany Park,2011,APRIL,9344.0,1841.0,9300.0
1,Albany Park,2011,AUGUST,11306.0,2434.0,11078.0
2,Albany Park,2011,DECEMBER,9934.0,1594.0,9609.0
3,Albany Park,2011,FEBRUARY,7023.0,1296.0,10500.0
4,Albany Park,2011,JANUARY,8427.0,1666.0,9604.0
...,...,...,...,...,...,...
12523,Wrightwood-Ashburn,2023,MARCH,834.0,806.0,6312.0
12524,Wrightwood-Ashburn,2023,MAY,NaN,NaN,NaN
12525,Wrightwood-Ashburn,2023,NOVEMBER,NaN,NaN,NaN
12526,Wrightwood-Ashburn,2023,OCTOBER,NaN,NaN,NaN


#### Finalize Metrics Dataset

In [13]:
# Map month names to numerical values for sorting
final_df["MONTH"] = final_df["MONTH"].apply(lambda m: months[m])

In [14]:
# Sort values
final_df = final_df.sort_values(by=["BRANCH_NAME", "YEAR", "MONTH"])

In [15]:
# Drop rows representing future (unknown values)
# NOTE: At the time of writing, values are available through April 2023
final_df = final_df.query("~(YEAR == 2023 & MONTH >= 5)")

In [16]:
# Rename columns
final_df = final_df.rename(columns={"BRANCH_NAME": "BRANCH"})

In [17]:
# Preview DataFrame
final_df

TYPE,BRANCH,YEAR,MONTH,CIRCULATION,COMPUTER_SESSIONS,VISITORS
4,Albany Park,2011,1,8427.0,1666.0,9604.0
3,Albany Park,2011,2,7023.0,1296.0,10500.0
7,Albany Park,2011,3,9702.0,1735.0,9050.0
0,Albany Park,2011,4,9344.0,1841.0,9300.0
8,Albany Park,2011,5,8865.0,1683.0,8271.0
...,...,...,...,...,...,...
12506,Wrightwood-Ashburn,2022,12,575.0,511.0,1410.0
12520,Wrightwood-Ashburn,2023,1,658.0,577.0,1625.0
12519,Wrightwood-Ashburn,2023,2,683.0,561.0,12146.0
12523,Wrightwood-Ashburn,2023,3,834.0,806.0,6312.0


In [18]:
# Write DataFrame to output CSV file, dropping index
clean_dir = Path.joinpath(Path.cwd().parent, Path("data/clean"))
metrics_fpath = (clean_dir / "branch_metrics.csv").as_posix()
final_df.to_csv(metrics_fpath, index=False)

#### Clean Library Branch Names and Locations

In [19]:
# Read data file
fname = "Libraries_-_Locations___Contact_Information__and_Usual_Hours_of_Operation.csv"
branches_df = pd.read_csv(f"../data/raw/branches/{fname}")

# Preview DataFrame
branches_df

,NAME,HOURS OF OPERATION,ADDRESS,CITY,STATE,ZIP,PHONE,WEBSITE,LOCATION
0,Vodak-East Side,"Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...",3710 E. 106th St.,Chicago,IL,60617,(312) 747-5500,https://www.chipublib.org/locations/71/,"(41.70283443594318, -87.61428978448026)"
1,Albany Park,"Mon. & Wed., 10-6; Tues. & Thurs., Noon-8; Fri...",3401 W. Foster Ave.,Chicago,IL,60625,(773) 539-5450,https://www.chipublib.org/locations/3/,"(41.97557881655979, -87.71361314512697)"
2,Avalon,"Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...",8148 S. Stony Island Ave.,Chicago,IL,60617,(312) 747-5234,https://www.chipublib.org/locations/8/,"(41.746393038286826, -87.5860053710736)"
3,Brainerd,"Mon. & Wed., 10-6; Tues. & Thurs., Noon-8; Fri...",1350 W. 89th St.,Chicago,IL,60620,(312) 747-6291,https://www.chipublib.org/locations/13/,"(41.73244482025524, -87.65772892721816)"
4,Popular Library at Water Works,"Mon. - Thurs., 10-6; Fri. & Sat., 9-5; Sun., 1-5",163 E. Pearson St.,Chicago,IL,60611,(312) 742-8811,https://www.chipublib.org/locations/73/,"(41.897484072390675, -87.62337776811282)"
...,...,...,...,...,...,...,...,...,...
76,Woodson Regional Library,"Mon. - Thurs., 9-8; Fri. & Sat., 9-5; Sun., 1-5",9525 S. Halsted St.,Chicago,IL,60628,(312) 747-6900,https://www.chipublib.org/locations/81/,"(41.720694885749005, -87.64304817213312)"
77,Mayfair,"Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...",4400 W. Lawrence Ave.,Chicago,IL,60630,(312) 744-1254,https://www.chipublib.org/locations/49/,"(41.968242773953044, -87.737968778247)"
78,Chicago Bee,"Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...",3647 S. State St.,Chicago,IL,60609,(312) 747-6872,https://www.chipublib.org/locations/18/,"(41.82824306445502, -87.6263495444489)"
79,Uptown,"Mon. & Wed., 10-6; Tues. & Thurs., Noon-8; Fri...",929 W. Buena Ave.,Chicago,IL,60613,(312) 744-8400,https://www.chipublib.org/locations/70/,"(41.95832305807637, -87.65424744448335)"


In [20]:
# Correct name misspellings and inconsistencies
branches_df = branches_df.rename(columns={"NAME ": "NAME"})
to_replace = {
    "Daley, Richard J.": "Daley, Richard J. - Bridgeport",
    "Daley, Richard M.": "Daley, Richard M. - W. Humboldt",
    "Harold Washtington Library Center": "Harold Washington Library Center",
    "Sulzer Regional Library": "Sulzer Regional",
    "Popular Library at Water Works": "Water Works"
}
replace_func = lambda n: to_replace[n] if n in to_replace.keys() else n
branches_df["NAME"] = branches_df["NAME"].apply(replace_func)

In [21]:
# Set a new boolean column to indicate whether
# the library branch is permanently closed.
branches_df["PERMANENTLY_CLOSED"] = 0

In [22]:
# Manually add missing Roosevelt branch
# NOTE: In 2019, Roosevelt moved from 1101 W. Taylor Street to 
# 1336 W. Taylor Street and reopened as the Little Italy Branch as
# part of a partnership with CPL and the Chicago Housing Authority
# to co-locate housing with library services.
# See https://www.chipublib.org/about-little-italy-branch/
roosevelt = {
    "NAME": "Roosevelt",
    "HOURS OF OPERATION": np.nan,
    "ADDRESS": "1101 W. Taylor Street",
    "CITY": "Chicago",
    "STATE": "IL",
    "ZIP": "60607",
    "PHONE": np.nan,
    "WEBSITE": np.nan,
    "LOCATION": "(41.86943838326072, -87.6544586134924)",
    "PERMANENTLY_CLOSED": 1
}
branches_df = branches_df.append(roosevelt, ignore_index=True)

# Preview row in DataFrame
branches_df.tail(1)

/tmp/ipykernel_5644/3503567889.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  branches_df = branches_df.append(roosevelt, ignore_index=True)


,NAME,HOURS OF OPERATION,ADDRESS,CITY,STATE,ZIP,PHONE,WEBSITE,LOCATION,PERMANENTLY_CLOSED
81,Roosevelt,NaN,1101 W. Taylor Street,Chicago,IL,60607,NaN,NaN,"(41.86943838326072, -87.6544586134924)",1


In [23]:
# Rename columns for consistency with metrics dataset
branches_df = branches_df.rename(columns={"NAME": "BRANCH"})

In [24]:
# Sort and preview DataFrame
branches_df = branches_df.sort_values(by="BRANCH")
branches_df

,BRANCH,HOURS OF OPERATION,ADDRESS,CITY,STATE,ZIP,PHONE,WEBSITE,LOCATION,PERMANENTLY_CLOSED
1,Albany Park,"Mon. & Wed., 10-6; Tues. & Thurs., Noon-8; Fri...",3401 W. Foster Ave.,Chicago,IL,60625,(773) 539-5450,https://www.chipublib.org/locations/3/,"(41.97557881655979, -87.71361314512697)",0
17,Altgeld,"Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...",955 E. 131st Street,Chicago,IL,60827,(312) 747-3270,https://www.chipublib.org/locations/4/,"(41.65719847558056, -87.59883829075173)",0
33,Archer Heights,"Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...",5055 S. Archer Ave.,Chicago,IL,60632,(312) 747-9241,https://www.chipublib.org/locations/5/,"(41.80110836194246, -87.72648385568911)",0
51,Austin,"Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...",5615 W. Race Ave.,Chicago,IL,60644,(312) 746-5038,https://www.chipublib.org/locations/6/,"(41.88917312206642, -87.7658203582574)",0
75,Austin-Irving,"Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...",6100 W. Irving Park Rd.,Chicago,IL,60634,(312) 744-6222,https://www.chipublib.org/locations/7/,"(41.95312267684315, -87.77928489355646)",0
...,...,...,...,...,...,...,...,...,...,...
47,West Pullman,"Mon. & Wed., Noon-8; Tue. & Thu., 10-6; Fri. &...",830 W. 119th St.,Chicago,IL,60643,(312) 747-1425,https://www.chipublib.org/locations/78/,"(41.67790647382097, -87.6431683153105)",0
29,West Town,"Mon. & Wed., 10-6; Tues. & Thurs., Noon-8; Fri...",1625 W. Chicago Ave.,Chicago,IL,60622,(312) 743-0450,https://www.chipublib.org/locations/79/,"(41.89594730478282, -87.66829305460979)",0
14,"Whitney M. Young, Jr.","Mon. & Wed., 10-6; Tues. & Thurs., Noon-8; Fri...",415 East 79th St.,Chicago,IL,60619,(312) 747-0039,https://www.chipublib.org/locations/80/,"(41.7510184591363, -87.61495065361933)",0
76,Woodson Regional Library,"Mon. - Thurs., 9-8; Fri. & Sat., 9-5; Sun., 1-5",9525 S. Halsted St.,Chicago,IL,60628,(312) 747-6900,https://www.chipublib.org/locations/81/,"(41.720694885749005, -87.64304817213312)",0


In [25]:
# Write DataFrame to output CSV file, dropping index
branches_fpath = (clean_dir / "branches.csv").as_posix()
branches_df.to_csv(branches_fpath, index=False)